In [4]:
import pandas as pd
import seaborn as sns

Link for this Data: https://www.eea.europa.eu/en/datahub/datahubitem-view/9405f714-8015-4b5b-a63c-280b82861b3d

In [5]:
file_name = "datasets/F1_4_Detailed releases at facility level with E-PRTR Sector and Annex I Activity detail into Air.csv"
data1 = pd.read_csv(file_name)
data1.head()

C:\Users\joaop\AppData\Local\Temp\ipykernel_11964\3702927021.py:2: DtypeWarning: Columns (7,10,16) have mixed types. Specify dtype option on import or set low_memory=False.
  data1 = pd.read_csv(file_name)


,countryName,EPRTRSectorCode,eprtrSectorName,EPRTRAnnexIMainActivityCode,EPRTRAnnexIMainActivityLabel,FacilityInspireID,facilityName,facilityNameConfidentialityReason,Longitude,Latitude,addressConfidentialityReason,City,targetRelease,pollutant,emissions,reportingYear,releasesConfidentialityReason
0,Austria,4.0,Chemical industry,4(d),Chemical installations for the production on a...,AT.CAED/9008390392737.FACILITY,Nufarm GmbH & CO KG,NaN,14.336056,48.290943,NaN,Linz,AIR,"1,2-dichloroethane (DCE-1,2)",3300.0,2007,NaN
1,Austria,9.0,Other activities,9(c),Installations for the surface treatment of sub...,AT.CAED/9008390396414.FACILITY,MAGNA STEYR Fahrzeugtechnik AG&Co KG,NaN,15.477778,47.019170,NaN,"Graz,07.Bez.:Liebenau",AIR,Non-methane volatile organic compounds (NMVOC),170000.0,2007,NaN
2,Austria,6.0,Paper and wood production and processing,6(b),Industrial plants for the production of paper ...,AT.CAED/9008390397176.FACILITY,Fritz EGGER GmbH & Co. OG,NaN,15.691833,48.259556,NaN,Unterradlberg,AIR,Carbon dioxide (CO2),228000000.0,2007,NaN
3,Austria,6.0,Paper and wood production and processing,6(b),Industrial plants for the production of paper ...,AT.CAED/9008390397176.FACILITY,Fritz EGGER GmbH & Co. OG,NaN,15.691833,48.259556,NaN,Unterradlberg,AIR,Nitrogen oxides (NOX),311000.0,2007,NaN
4,Austria,3.0,Mineral industry,3(c),Installations for the production of cement cli...,AT.CAED/9008390397220.FACILITY,voestalpine Stahl GmbH,NaN,14.159778,47.807083,NaN,Klaus an der Pyhrnbahn,AIR,Carbon dioxide (CO2),329000000.0,2007,NaN


In [6]:
# now choose only london
data1 = data1[data1["City"]== 'London']
data1
#região à volta

,countryName,EPRTRSectorCode,eprtrSectorName,EPRTRAnnexIMainActivityCode,EPRTRAnnexIMainActivityLabel,FacilityInspireID,facilityName,facilityNameConfidentialityReason,Longitude,Latitude,addressConfidentialityReason,City,targetRelease,pollutant,emissions,reportingYear,releasesConfidentialityReason
297134,United Kingdom,8.0,Animal and vegetable products from the food an...,8(b),Treatment and processing intended for the prod...,UK.CAED/EW_EA-3678.FACILITY,United Biscuits (UK) Limited T/A Pladis,NaN,-0.261731,51.535900,NaN,London,AIR,Hydrochlorofluorocarbons (HCFCs),15.0,2009,NaN
300011,United Kingdom,8.0,Animal and vegetable products from the food an...,8(b),Treatment and processing intended for the prod...,UK.CAED/EW_EA-3678.FACILITY,United Biscuits (UK) Limited T/A Pladis,NaN,-0.261731,51.535900,NaN,London,AIR,Hydrochlorofluorocarbons (HCFCs),295.0,2010,NaN
302842,United Kingdom,8.0,Animal and vegetable products from the food an...,8(b),Treatment and processing intended for the prod...,UK.CAED/EW_EA-3678.FACILITY,United Biscuits (UK) Limited T/A Pladis,NaN,-0.261732,51.535900,NaN,London,AIR,Hydrochlorofluorocarbons (HCFCs),191.0,2011,NaN
304094,United Kingdom,8.0,Animal and vegetable products from the food an...,8(b),Treatment and processing intended for the prod...,GB.EEA/13486.FACILITY,"Tate And Lyle Plc, Tate & Lyle",NaN,0.047184,51.501500,NaN,London,AIR,Carbon dioxide (CO2),106000000.0,2012,NaN
305636,United Kingdom,8.0,Animal and vegetable products from the food an...,8(b),Treatment and processing intended for the prod...,UK.CAED/EW_EA-3678.FACILITY,United Biscuits (UK) Limited T/A Pladis,NaN,-0.261720,51.535900,NaN,London,AIR,Hydrochlorofluorocarbons (HCFCs),16.5,2012,NaN
306887,United Kingdom,8.0,Animal and vegetable products from the food an...,8(b),Treatment and processing intended for the prod...,GB.EEA/13486.FACILITY,"Tate And Lyle Plc, Tate & Lyle",NaN,0.047184,51.501500,NaN,London,AIR,Carbon dioxide (CO2),111000000.0,2013,NaN
307962,United Kingdom,5.0,Waste and wastewater management,5(b),Installations for the incineration of non-haza...,UK.CAED/EW_EA-16639.FACILITY,Crossness Sludge Powered Generator,NaN,0.146927,51.505300,NaN,London,AIR,Nitrous oxide (N2O),31100.0,2013,NaN
308468,United Kingdom,8.0,Animal and vegetable products from the food an...,8(b),Treatment and processing intended for the prod...,UK.CAED/EW_EA-3678.FACILITY,United Biscuits (UK) Limited T/A Pladis,NaN,-0.261720,51.535900,NaN,London,AIR,Ammonia (NH3),178000.0,2013,NaN
308469,United Kingdom,8.0,Animal and vegetable products from the food an...,8(b),Treatment and processing intended for the prod...,UK.CAED/EW_EA-3678.FACILITY,United Biscuits (UK) Limited T/A Pladis,NaN,-0.261720,51.535900,NaN,London,AIR,Hydrochlorofluorocarbons (HCFCs),254.0,2013,NaN
308470,United Kingdom,8.0,Animal and vegetable products from the food an...,8(b),Treatment and processing intended for the prod...,UK.CAED/EW_EA-3678.FACILITY,United Biscuits (UK) Limited T/A Pladis,NaN,-0.261720,51.535900,NaN,London,AIR,Methane (CH4),226000.0,2013,NaN


In [7]:
#select relevant data
data1 = data1.loc[:,["targetRelease","pollutant","emissions","reportingYear"]]
data1.sort_values('reportingYear',inplace=True)
data1

,targetRelease,pollutant,emissions,reportingYear
297134,AIR,Hydrochlorofluorocarbons (HCFCs),15.0,2009
300011,AIR,Hydrochlorofluorocarbons (HCFCs),295.0,2010
302842,AIR,Hydrochlorofluorocarbons (HCFCs),191.0,2011
304094,AIR,Carbon dioxide (CO2),106000000.0,2012
305636,AIR,Hydrochlorofluorocarbons (HCFCs),16.5,2012
306887,AIR,Carbon dioxide (CO2),111000000.0,2013
307962,AIR,Nitrous oxide (N2O),31100.0,2013
308468,AIR,Ammonia (NH3),178000.0,2013
308469,AIR,Hydrochlorofluorocarbons (HCFCs),254.0,2013
308470,AIR,Methane (CH4),226000.0,2013


In [8]:
#health index
file_name = "datasets/air-quality-london-monthly-averages.csv"
data2 = pd.read_csv(file_name)
data2

,Month,London Mean Roadside:Nitric Oxide (ug/m3),London Mean Roadside:Nitrogen Dioxide (ug/m3),London Mean Roadside:Oxides of Nitrogen (ug/m3),London Mean Roadside:Ozone (ug/m3),London Mean Roadside:PM10 Particulate (ug/m3),London Mean Roadside:PM2.5 Particulate (ug/m3),London Mean Roadside:Sulphur Dioxide (ug/m3),London Mean Background:Nitric Oxide (ug/m3),London Mean Background:Nitrogen Dioxide (ug/m3),London Mean Background:Oxides of Nitrogen (ug/m3),London Mean Background:Ozone (ug/m3),London Mean Background:PM10 Particulate (ug/m3),London Mean Background:PM2.5 Particulate (ug/m3),London Mean Background:Sulphur Dioxide (ug/m3)
0,Jan-08,NaN,55.5,NaN,29.5,25.0,14.7,4.2,NaN,42.3,NaN,36.9,18.8,.,3.6
1,Feb-08,NaN,75.9,NaN,20.3,39.5,28.8,7.6,NaN,60.2,NaN,26.4,31.9,.,6.7
2,Mar-08,NaN,55.6,NaN,40.1,21.6,12.3,3.9,NaN,39.8,NaN,50.2,15.5,.,2.3
3,Apr-08,NaN,61.8,NaN,37.9,28.7,20.5,4.5,NaN,44.0,NaN,50.1,21.7,.,3.2
4,May-08,NaN,62.9,NaN,46.3,34.6,27.5,4.6,NaN,44.1,NaN,60.5,29.5,16.6,4.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,Mar-19,31.0,42.6,90.0,34.9,20.9,11.3,4.9,7.7,25.7,36.0,52.7,16.6,10.9,2.3
135,Apr-19,25.2,46.2,84.9,38.1,32.3,22.8,12.4,6.9,31.5,39.9,54.7,29.2,23.1,2.8
136,May-19,24.7,39.7,77.5,35.7,19.5,10.7,11.5,5.5,22.4,29.3,53.7,15.3,10.9,1.5
137,Jun-19,22.0,34.9,68.6,31.4,18.0,9.2,4.5,5.0,19.4,24.9,51.8,14.1,9.5,1.8


In [9]:
# now choose only london
data2 = data2.loc[:,["Month","London Mean Roadside:Nitrogen Dioxide (ug/m3)","London Mean Roadside:PM2.5 Particulate (ug/m3)"]]
data2

,Month,London Mean Roadside:Nitrogen Dioxide (ug/m3),London Mean Roadside:PM2.5 Particulate (ug/m3)
0,Jan-08,55.5,14.7
1,Feb-08,75.9,28.8
2,Mar-08,55.6,12.3
3,Apr-08,61.8,20.5
4,May-08,62.9,27.5
...,...,...,...
134,Mar-19,42.6,11.3
135,Apr-19,46.2,22.8
136,May-19,39.7,10.7
137,Jun-19,34.9,9.2


In [10]:
data2['Month'] = pd.to_datetime(data2['Month'], format='%b-%y')
data2['Year'] = data2['Month'].dt.year

# Agrupe por ano e calcule a média das colunas 2 e 3
data2 = data2.groupby('Year')[['London Mean Roadside:Nitrogen Dioxide (ug/m3)', 'London Mean Roadside:PM2.5 Particulate (ug/m3)']].mean().reset_index()
data2


,Year,London Mean Roadside:Nitrogen Dioxide (ug/m3),London Mean Roadside:PM2.5 Particulate (ug/m3)
0,2008,57.016667,17.633333
1,2009,58.333333,17.033333
2,2010,58.633333,17.441667
3,2011,55.866667,17.983333
4,2012,60.069231,16.300000
5,2013,58.218182,16.863636
6,2014,56.675000,15.275000
7,2015,54.466667,13.941667
8,2016,56.375000,14.591667
9,2017,51.233333,13.216667


In [11]:
data2 = data2.rename(columns={"Year":"reportingYear"})
data2 = data2.rename(columns={"London Mean Roadside:Nitrogen Dioxide (ug/m3)":"NO2"})
data2 = data2.rename(columns={"London Mean Roadside:PM2.5 Particulate (ug/m3)":"PM25"})
data2


,reportingYear,NO2,PM25
0,2008,57.016667,17.633333
1,2009,58.333333,17.033333
2,2010,58.633333,17.441667
3,2011,55.866667,17.983333
4,2012,60.069231,16.300000
5,2013,58.218182,16.863636
6,2014,56.675000,15.275000
7,2015,54.466667,13.941667
8,2016,56.375000,14.591667
9,2017,51.233333,13.216667


Link for this data: https://roadtraffic.dft.gov.uk/regions/6

In [12]:
file_name = "datasets/dft_aadf_region_id_6.csv"
data3 = pd.read_csv(file_name)
data3

,count_point_id,year,region_id,region_name,local_authority_id,local_authority_name,road_name,road_type,start_junction_road_name,end_junction_road_name,...,buses_and_coaches,lgvs,hgvs_2_rigid_axle,hgvs_3_rigid_axle,hgvs_4_or_more_rigid_axle,hgvs_3_or_4_articulated_axle,hgvs_5_articulated_axle,hgvs_6_articulated_axle,all_hgvs,all_motor_vehicles
0,6750,2014,6,London,96,Islington,A201,Major,Swinton St,Pentonville Rd,...,787,1728,204,74,132,15,6,18,449,11916
1,8352,2014,6,London,104,Lewisham,A2,Major,A20,A20,...,430,1415,258,36,58,6,11,22,390,9426
2,6807,2014,6,London,178,Kingston upon Thames,A240,Major,A3210,A307,...,175,1589,251,35,13,2,4,19,325,12951
3,27765,2014,6,London,57,Barnet,A5109,Major,Barnet Lane,A1000/A109,...,302,1651,246,12,12,0,1,1,273,19276
4,8488,2014,6,London,104,Lewisham,A20,Major,A20 Amersham Rd,A2 Parkfield Rd,...,656,1617,286,83,27,5,8,7,415,12821
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50184,37106,2022,6,London,66,Hillingdon,A404,Major,LA Boundary,A4180,...,447,1672,211,42,43,9,7,6,318,16531
50185,17099,2022,6,London,66,Hillingdon,A437,Major,A4(T),A312,...,553,1628,120,17,5,1,13,5,162,15900
50186,17024,2022,6,London,66,Hillingdon,A408,Major,A4,A3044,...,248,687,75,3,3,4,0,12,97,6503
50187,37713,2022,6,London,66,Hillingdon,A4007,Major,LA Boundary,A4020/A408,...,121,2078,364,78,24,5,66,53,589,12357


In [13]:
#select relevant data
data3 = data3.loc[:,["year","all_motor_vehicles"]]
data3

,year,all_motor_vehicles
0,2014,11916
1,2014,9426
2,2014,12951
3,2014,19276
4,2014,12821
...,...,...
50184,2022,16531
50185,2022,15900
50186,2022,6503
50187,2022,12357


In [14]:
#sum the number of total vehicles by year
data3 = data3.groupby('year')['all_motor_vehicles'].sum().reset_index()
data3 = data3.rename(columns={"year":"reportingYear"})
data3

,reportingYear,all_motor_vehicles
0,2000,55572506
1,2001,55553417
2,2002,54356983
3,2003,55230556
4,2004,54561589
5,2005,54435671
6,2006,55846934
7,2007,54834530
8,2008,55040271
9,2009,54216130


In [15]:
#health index
file_name = "health-index-1.csv"
data4 = pd.read_csv(file_name)
data4


,id,utc_created,utc_updated,country,country_code,indicator_code,indicator,value,year
0,1,2016-01-07 10:10:52.689964,2016-01-07 10:10:52.690139,Afghanistan,AFG,72206,HDI: Health index,0.327,1980
1,2,2016-01-07 10:10:52.689964,2016-01-07 10:10:52.690139,Afghanistan,AFG,72206,HDI: Health index,0.375,1985
2,3,2016-01-07 10:10:52.689964,2016-01-07 10:10:52.690139,Afghanistan,AFG,72206,HDI: Health index,0.440,1990
3,4,2016-01-07 10:10:52.689964,2016-01-07 10:10:52.690139,Afghanistan,AFG,72206,HDI: Health index,0.498,1995
4,5,2016-01-07 10:10:52.689964,2016-01-07 10:10:52.690139,Afghanistan,AFG,72206,HDI: Health index,0.536,2000
...,...,...,...,...,...,...,...,...,...
2477,2478,2016-01-07 10:10:52.689964,2016-01-07 10:10:52.690139,Zimbabwe,ZWE,72206,HDI: Health index,0.481,2009
2478,2479,2016-01-07 10:10:52.689964,2016-01-07 10:10:52.690139,Zimbabwe,ZWE,72206,HDI: Health index,0.518,2010
2479,2480,2016-01-07 10:10:52.689964,2016-01-07 10:10:52.690139,Zimbabwe,ZWE,72206,HDI: Health index,0.554,2011
2480,2481,2016-01-07 10:10:52.689964,2016-01-07 10:10:52.690139,Zimbabwe,ZWE,72206,HDI: Health index,0.587,2012


In [16]:
data4 = data4[data4["country"]== 'United Kingdom']
data4


,id,utc_created,utc_updated,country,country_code,indicator_code,indicator,value,year
793,794,2016-01-07 10:10:52.689964,2016-01-07 10:10:52.690139,United Kingdom,GBR,72206,HDI: Health index,0.823,1980
794,795,2016-01-07 10:10:52.689964,2016-01-07 10:10:52.690139,United Kingdom,GBR,72206,HDI: Health index,0.839,1985
795,796,2016-01-07 10:10:52.689964,2016-01-07 10:10:52.690139,United Kingdom,GBR,72206,HDI: Health index,0.855,1990
796,797,2016-01-07 10:10:52.689964,2016-01-07 10:10:52.690139,United Kingdom,GBR,72206,HDI: Health index,0.871,1995
797,798,2016-01-07 10:10:52.689964,2016-01-07 10:10:52.690139,United Kingdom,GBR,72206,HDI: Health index,0.888,2000
798,799,2016-01-07 10:10:52.689964,2016-01-07 10:10:52.690139,United Kingdom,GBR,72206,HDI: Health index,0.908,2005
799,800,2016-01-07 10:10:52.689964,2016-01-07 10:10:52.690139,United Kingdom,GBR,72206,HDI: Health index,0.912,2006
800,801,2016-01-07 10:10:52.689964,2016-01-07 10:10:52.690139,United Kingdom,GBR,72206,HDI: Health index,0.916,2007
801,802,2016-01-07 10:10:52.689964,2016-01-07 10:10:52.690139,United Kingdom,GBR,72206,HDI: Health index,0.922,2009
802,803,2016-01-07 10:10:52.689964,2016-01-07 10:10:52.690139,United Kingdom,GBR,72206,HDI: Health index,0.924,2010


In [17]:
data4 = data4.loc[:,["value","year"]]
data4.sort_values('year',inplace=True)
data4 = data4.rename(columns={"year":"reportingYear"})
data4

,value,reportingYear
793,0.823,1980
794,0.839,1985
795,0.855,1990
796,0.871,1995
797,0.888,2000
798,0.908,2005
799,0.912,2006
800,0.916,2007
801,0.922,2009
802,0.924,2010


In [18]:
#first merge data1 and data2
merged1 = data1.merge(data2, on="reportingYear", how="outer")
merged2 = merged1.merge(data3, on="reportingYear", how="outer")
mergedFinal = merged2.merge(data4, on="reportingYear", how="outer")
mergedFinal.sort_values('reportingYear',inplace=True)
mergedFinal = mergedFinal.rename(columns={"value":"healthValue"})
mergedFinal.set_index("reportingYear", inplace=True)
all_years = mergedFinal.index

mergedFinal

,targetRelease,pollutant,emissions,NO2,PM25,all_motor_vehicles,healthValue
reportingYear,,,,,,,
1980,NaN,NaN,NaN,NaN,NaN,NaN,0.823
1985,NaN,NaN,NaN,NaN,NaN,NaN,0.839
1990,NaN,NaN,NaN,NaN,NaN,NaN,0.855
1995,NaN,NaN,NaN,NaN,NaN,NaN,0.871
2000,NaN,NaN,NaN,NaN,NaN,55572506.0,0.888
2001,NaN,NaN,NaN,NaN,NaN,55553417.0,NaN
2002,NaN,NaN,NaN,NaN,NaN,54356983.0,NaN
2003,NaN,NaN,NaN,NaN,NaN,55230556.0,NaN
2004,NaN,NaN,NaN,NaN,NaN,54561589.0,NaN


In [22]:
#https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/healthandwellbeing/datasets/healthindexunderlyingdataengland

data = pd.read_csv("datasets/healthindexunderlyingdataenglandcanscreenfix.csv",delimiter=";")
data


,Area code,Area name,Value,Numerator,Denominator,Indicator name,Year
0,E06000001,Hartlepool,"62,68947549",NaN,NaN,Physical activity,2015
1,E06000002,Middlesbrough,"62,8371136",NaN,NaN,Physical activity,2015
2,E06000003,Redcar and Cleveland,"65,80605787",NaN,NaN,Physical activity,2015
3,E06000004,Stockton-on-Tees,"59,83048525",NaN,NaN,Physical activity,2015
4,E06000005,Darlington,"65,39884288",NaN,NaN,Physical activity,2015
...,...,...,...,...,...,...,...
120339,E09000029,Sutton,833,NaN,NaN,Workplace safety,2021
120340,E09000030,Tower Hamlets,111,NaN,NaN,Workplace safety,2021
120341,E09000031,Waltham Forest,280,NaN,NaN,Workplace safety,2021
120342,E09000032,Wandsworth,279,NaN,NaN,Workplace safety,2021


1.delimitar as cidades por raio, não considerar apenas londres  
2.visualização para mostrar dados incompletos ou em falha  
3.justificar preenchimento de nulls e remoção de certas variáveis  